# Tensorboard for keras CNN

# 1. load data

In [2]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split


digit = load_digits()

data_x = digit.data
data_y = digit.target


x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2, random_state=42)


# 2. keras preprocess

In [3]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 8, 8


if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape: (1437, 8, 8, 1)
1437 train samples
360 test samples


# 3. CNN model

In [4]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


# 4. training with tensorborad data support

In [14]:
# call back for tensorboard
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph_CNN', 
                                         write_graph=True, 
                                         write_images=True, 
                                         embeddings_layer_names=None, 
                                         embeddings_metadata=None)


model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
         callbacks=[tbCallBack])


Train on 1437 samples, validate on 360 samples
Epoch 1/12
1437/1437 [==============================] - 0s - loss: 0.4529 - acc: 0.8476 - val_loss: 0.2976 - val_acc: 0.8972
Epoch 2/12
1437/1437 [==============================] - 0s - loss: 0.4493 - acc: 0.8566 - val_loss: 0.2548 - val_acc: 0.9278
Epoch 3/12
1437/1437 [==============================] - 0s - loss: 0.4217 - acc: 0.8713 - val_loss: 0.2375 - val_acc: 0

# 5. start tensorboard

In [15]:
! tensorboard --logdir Graph_CNN/

Starting TensorBoard b'41' on port 6006
(You can navigate to http://127.0.1.1:6006)
